In [ ]:
from pathlib import Path
from collections import defaultdict
from PIL import Image
import numpy as np


In [ ]:
import plotly.offline as ply
ply.init_notebook_mode(connected=True)

In [ ]:
conversions = defaultdict(dict)
def conversion(src, dst):
    def register(fn):
        conversions[src][dst] = fn
        return fn
    return register
    
def convert(data, chain):
    sources = chain[:-1]
    dests = chain[1:]
    for src, dst in zip(sources, dests):
        data = conversions[src][dst](data)
    return data

In [ ]:
def parse_matrix(matstr):
    rows = matstr.strip().split("\n")
    mat = [[float(n) for n in row.split()] for row in rows]
    return np.array(mat)

In [ ]:
rgb2xyz_matrix_str = """
 0.4124564  0.3575761  0.1804375
 0.2126729  0.7151522  0.0721750
 0.0193339  0.1191920  0.9503041
"""

rgb2xyz_matrix = parse_matrix(rgb2xyz_matrix_str)

def invcompand_sRGB(V):
    return np.where(V <= 0.04045, V / 12.92, ((V+0.055)/1.055)**2.4)

@conversion("sRGB", "XYZ")
def sRGB_to_XYZ(sRGB):
    RGB = invcompand_sRGB(sRGB / 255.0)
    XYZ = (RGB.reshape([-1, 3]) @ rgb2xyz_matrix.T).reshape(sRGB.shape)
    return XYZ

In [ ]:
def compand_sRGB(v):
    return np.where(v <= 0.0031308, 12.92*v, 1.055 * v**(1/2.4) - 0.055)

XYZ_to_sRGB_matrix_str = """
 3.2404542 -1.5371385 -0.4985314
-0.9692660  1.8760108  0.0415560
 0.0556434 -0.2040259  1.0572252
"""

XYZ_to_sRGB_matrix = parse_matrix(XYZ_to_sRGB_matrix_str)

@conversion("XYZ", "sRGB")
def XYZ_to_sRGB(XYZ):
    RGB = (XYZ.reshape([-1, 3]) @ XYZ_to_sRGB_matrix.T).reshape(XYZ.shape)
    sRGB = compand_sRGB(RGB)
    return sRGB

In [ ]:
@conversion("xyY", "XYZ")
def xyY_to_XYZ(xyY):
    x = xyY[..., 0]
    y = xyY[..., 1]
    Y = xyY[..., 2]
    X = x/y * Y
    Z = (1-x-y)/y * Y
    return np.stack([X, Y, Z], axis=-1)

@conversion("XYZ", "xyY")
def XYZ_to_xyY(XYZ):
    X, Y, Z = XYZ[..., 0], XYZ[..., 1], XYZ[..., 2]
    x = X / (X+Y+Z)
    y = Y / (X+Y+Z)
    x[np.isnan(x)] = 0.33
    y[np.isnan(y)] = 0.33
    return np.stack([x, y, Y], axis=-1)

In [ ]:
D65_xyY = np.array([[0.31271, 0.32902, 1]])
D65_XYZ = XYZ_to_xyY(D65_xyY)

@conversion("XYZ", "LUV_D65")
def XYZ_to_LUV_D65(XYZ):
    epsilon = 216/24389
    kappa = 24389/27
    X, Y, Z = XYZ[..., 0], XYZ[..., 1], XYZ[..., 2]
    X_r, Y_r, Z_r = D65_XYZ[..., 0], D65_XYZ[..., 1], D65_XYZ[..., 2]
    u1 = 4*X/(X+15*Y+3*Z)
    v1 = 9*Y/(X+15*Y+3*Z)
    u1_r = 4*X_r/(X_r+15*Y_r+3*Z_r)
    v1_r = 9*Y_r/(X_r+15*Y_r+3*Z_r)
    yr = Y/Y_r
    L = np.where(y_r > epsilon, 116 * y_r**(1/3)-16, kappa*y_r)
    u = 13*L*(u1-u1_r)
    v = 13*L*(v1-v1_r)
    return np.stack([L, u, v], axis=-1)

In [ ]:
image_paths = list(Path("mastodon").glob("*"))
image_paths

In [ ]:
import random
idx = random.choice(range(len(image_paths)))
image = Image.open(image_paths[idx])

image

In [ ]:
xyY = convert(sRGB, ["sRGB", "XYZ", "xyY"])

In [ ]:
from KDEpy import FFTKDE
kde = FFTKDE(bw=0.005)
points = np.stack([xyY[..., 0].flatten(), xyY[..., 1].flatten()], axis=-1)
kde.fit(points)
range_y = np.linspace(0, 0.65, 256)
range_x = np.linspace(0.10, 0.75, 256)
grid_y, grid_x = np.meshgrid(range_y, range_x)
grid = np.stack([grid_x.flatten(), grid_y.flatten()], axis=-1)
pdf = kde.evaluate(grid)

grid = grid.reshape([*grid_x.shape, 2])
pdf = pdf.reshape(grid_x.shape)

grid_xyY = np.block([grid, 0.6*np.ones((*grid.shape[:-1], 1))])
grid_XYZ = xyY_to_XYZ(grid_xyY)
grid_sRGB = XYZ_to_sRGB(grid_XYZ)

invalid_sRGB = (np.min(grid_sRGB, axis=-1) < 0) | (np.max(grid_sRGB, axis=-1) > 1)
grid_sRGB[invalid_sRGB] = 1

In [ ]:
thres = np.quantile(pdf, 0.99)
norm_pdf = np.where(pdf <= thres, pdf / thres, 1)
sRGBA = np.block([grid_sRGB, norm_pdf[..., None]])
sRGBA = np.swapaxes(sRGBA, 0, 1)[:1:-1]
chroma_image = Image.fromarray(np.ascontiguousarray(sRGBA*255, dtype=np.uint8))
chroma_image